In [2]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout

In [5]:
train_ds=keras.utils.image_dataset_from_directory(
    directory= '/content/drive/MyDrive/PetImages/PetImages/train',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256,256)

)
test_ds=keras.utils.image_dataset_from_directory(
    directory= '/content/drive/MyDrive/PetImages/PetImages/test',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256,256)

)

Found 557 files belonging to 2 classes.
Found 140 files belonging to 2 classes.


In [6]:
#normalize
def process(image,lebel):
     image =tf.cast(image/255.,tf.float32)
     return image,lebel

train_ds=train_ds.map(process)
test_ds=test_ds.map(process)

In [8]:
#create CNN model
model=Sequential()
model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,3)))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))
model.add(BatchNormalization())

model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))
model.add(BatchNormalization())

model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))
model.add(BatchNormalization())

# Example of adding dropout
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(64,activation='sigmoid'))
model.add(Dropout(0.25))
model.add(Dense(1,activation='sigmoid'))

In [9]:

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 127, 127, 32)      0         
 g2D)                                                            
                                                                 
 batch_normalization_3 (Bat  (None, 127, 127, 32)      128       
 chNormalization)                                                
                                                                 
 conv2d_4 (Conv2D)           (None, 125, 125, 32)      9248      
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 62, 62, 32)        0         
 g2D)                                                            
                                                      

In [10]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [11]:
hist=model.fit(train_ds,epochs=17,validation_data=test_ds)

Epoch 1/17
18/18 [==============================] - 218s 8s/step - loss: 0.6961 - accuracy: 0.5745 - val_loss: 0.7337 - val_accuracy: 0.5000
Epoch 2/17
18/18 [==============================] - 5s 165ms/step - loss: 0.6081 - accuracy: 0.6553 - val_loss: 0.8153 - val_accuracy: 0.5071
Epoch 3/17
18/18 [==============================] - 5s 202ms/step - loss: 0.5478 - accuracy: 0.7181 - val_loss: 0.8165 - val_accuracy: 0.5000
Epoch 4/17
18/18 [==============================] - 6s 253ms/step - loss: 0.5026 - accuracy: 0.7451 - val_loss: 1.0255 - val_accuracy: 0.5000
Epoch 5/17
18/18 [==============================] - 5s 168ms/step - loss: 0.4386 - accuracy: 0.7882 - val_loss: 1.0071 - val_accuracy: 0.5000
Epoch 6/17
18/18 [==============================] - 5s 200ms/step - loss: 0.3982 - accuracy: 0.8169 - val_loss: 0.7935 - val_accuracy: 0.5000
Epoch 7/17
18/18 [==============================] - 6s 248ms/step - loss: 0.3332 - accuracy: 0.8743 - val_loss: 0.8728 - val_accuracy: 0.5000
Epoch 8

In [12]:
hist=model.fit(train_ds,epochs=10,validation_data=test_ds)

Epoch 1/10
18/18 [==============================] - 5s 202ms/step - loss: 0.0175 - accuracy: 0.9982 - val_loss: 1.9037 - val_accuracy: 0.5357
Epoch 2/10
18/18 [==============================] - 6s 243ms/step - loss: 0.0151 - accuracy: 0.9982 - val_loss: 1.7751 - val_accuracy: 0.5643
Epoch 3/10
18/18 [==============================] - 5s 202ms/step - loss: 0.0114 - accuracy: 1.0000 - val_loss: 1.8583 - val_accuracy: 0.5714
Epoch 4/10
18/18 [==============================] - 6s 244ms/step - loss: 0.0103 - accuracy: 1.0000 - val_loss: 1.9847 - val_accuracy: 0.5714
Epoch 5/10
18/18 [==============================] - 5s 202ms/step - loss: 0.0095 - accuracy: 1.0000 - val_loss: 1.8656 - val_accuracy: 0.5714
Epoch 6/10
18/18 [==============================] - 4s 170ms/step - loss: 0.0123 - accuracy: 0.9982 - val_loss: 2.0433 - val_accuracy: 0.5857
Epoch 7/10
18/18 [==============================] - 6s 246ms/step - loss: 0.0149 - accuracy: 0.9964 - val_loss: 1.6075 - val_accuracy: 0.6286
Epoch 

In [13]:
from sklearn.metrics import f1_score
# Make predictions on the test dataset
predictions1 = model.predict(test_ds)

# Convert probabilities to binary predictions using a threshold (adjust as needed)
threshold = 0.5
binary_predictions1 = (predictions1 > threshold).astype(int)

# Extract true labels from 'test_ds'
true_labels1 = np.concatenate([y for x, y in test_ds], axis=0)

# Calculate F1 score
f1 = f1_score(true_labels1, binary_predictions1)

5/5 [==============================] - 1s 20ms/step


In [14]:
print(f1)

0.4070796460176991


In [16]:
from sklearn.metrics import accuracy_score
#accuracy on test dataset
accuracy = accuracy_score(true_labels1, binary_predictions1)
print(accuracy)

0.5214285714285715
